# COVbat TOP and StrokeMRI harmonized datasets

Note this must be run in the `covbat` environment. ENvironment file inside vendor/covbat

## import libraries

In [ ]:
import os
import sys

import pandas as pd
import numpy as np

import patsy

sys.path.insert(0, '../../') # path to functions

import cvasl.harmony as har
import cvasl.vendor.covbat.covbat as covbat

impo##rt data

In [ ]:
# Datasets for this work

MRI = pd.read_csv('../new_data/TrainingDataComplete_StrokeMRI.csv')
TOP = pd.read_csv('../new_data/TrainingDataComplete_TOP.csv')

In [ ]:
TOP = TOP.drop([ 'ID'], axis= 1)
TOP['Site'] = 0
TOP.head(3)

In [ ]:
MRI = MRI.drop(['ID'],axis = 1)
MRI.tail(5)

In [ ]:
to_be_harmonized_or_covar = [
    'Age', 'Sex', 'ACA_B_CoV', 'MCA_B_CoV', 'PCA_B_CoV','DeepWM_B_CoV',
     'ACA_B_CBF', 'MCA_B_CBF', 'PCA_B_CBF', 'LD', 'PLD','DeepWM_B_CBF',
       'Labelling', 'Readout', 'M0','TotalGM_B_CoV',
       'TotalGM_B_CBF',
]
not_harmonized= ['GM_vol', 'WM_vol', 'CSF_vol','GM_ICVRatio', 'GMWM_ICVRatio', 'WMHvol_WMvol', 'WMH_count',
                'LD', 'PLD', 'Labelling',
       'Readout', 'M0','DeepWM_B_CoV','DeepWM_B_CBF',]
TOP_semi_features = TOP.drop(to_be_harmonized_or_covar,axis=1)
MRI_semi_features = MRI.drop(to_be_harmonized_or_covar,axis=1) 

TOP = TOP.drop(not_harmonized,axis=1)
MRI = MRI.drop(not_harmonized,axis=1) 

In [ ]:
TOPMRI = pd.concat([TOP,MRI])

TOPMRI.head(3) 

In [ ]:
TOPMRI.tail(3)

In [ ]:
phenoTOPMRI = TOPMRI[['participant_id','Age', 'Sex', 'Site']]
phenoTOPMRI = phenoTOPMRI.set_index('participant_id')
phenoTOPMRI.head(3)

In [ ]:
dat_TOPMRI = TOPMRI.set_index('participant_id')
dat_TOPMRI = dat_TOPMRI.T 

In [ ]:
dat_TOPMRI.head(3)

In [ ]:
phenoTOPMRI.head(3)

In [ ]:
modZ = patsy.dmatrix("~ Age + Sex", phenoTOPMRI, return_type="dataframe")
modZ.head(3)

In [ ]:
covbatTOPMRI = covbat.combat(dat_TOPMRI.tail(10), phenoTOPMRI['Site'],  model=modZ, numerical_covariates ="Age")

In [ ]:
covbatTOPMRI = covbatTOPMRI[2:] 

In [ ]:
dat_TOPMRI.head(3)

In [ ]:
covbatTOPMRI = pd.concat([dat_TOPMRI.head(3), covbatTOPMRI])
covbatTOPMRI = covbatTOPMRI.T
covbatTOPMRI = covbatTOPMRI.reset_index()

In [ ]:
covbatTOPMRI

In [ ]:
TOP_adjusted = covbatTOPMRI[covbatTOPMRI['Site'] == 0]
MRI_adjusted = covbatTOPMRI[covbatTOPMRI['Site'] == 1] 

In [ ]:
TOP_adjusted = TOP_adjusted.drop('Site', axis=1)
TOP_adjusted.head(3)

In [ ]:
MRI_adjusted = MRI_adjusted.drop('Site', axis=1)
MRI_adjusted.tail(3)

In [ ]:
TOP_adjusted = TOP_adjusted.merge(TOP_semi_features,on='participant_id')
MRI_adjusted = MRI_adjusted.merge(MRI_semi_features,on='participant_id')

In [ ]:
MRI_adjusted = MRI_adjusted.drop('Site', axis=1)
TOP_adjusted = TOP_adjusted.drop('Site', axis=1)

In [ ]:
TOP_adjusted.to_csv('harm_results/covbat/top_covbat_a_mri.csv')
MRI_adjusted.to_csv('harm_results/covbat/mri_covbat_a_top.csv')

In [ ]:
TOP_adjusted.shape